In [11]:
from pathlib import Path

root_results_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences"
# Define the directory to search
directory = Path(root_results_path)

# Use rglob to search recursively ('**/*.csv') 
# or glob for just the current folder ('*.csv')
csv_files = []
for path in list(directory.rglob('*.csv')):
    if "tests" in str(path) or len(str(path).split('/')[-1])<20:
        continue
    csv_files.append(path)

root_results_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences_postprocess"
# Define the directory to search
directory = Path(root_results_path)

# Use rglob to search recursively ('**/*.csv') 
# or glob for just the current folder ('*.csv')z
for path in list(directory.rglob('*.csv')):
    if "tests" in str(path) or len(str(path).split('/')[-1])<20:
        continue
    csv_files.append(path)

csv_files

[PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/resume_results_old.csv'),
 PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final/TTA/overlap_0.5/th_0.4/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final_TTA_overlap_0.5_th_0.4.csv'),
 PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final/TTA/overlap_0.5/th_0.5/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final_TTA_overlap_0.5_th_0.5.csv'),
 PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final/TTA/overlap_0.5/th_0.6/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final_TTA_overlap_0.5_th_0.6.csv'),
 PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final/no_TTA/overlap_0.5/th_0.4/CESoftDiceclDiceLoss_step3_1_DAsa

In [12]:
import pandas as pd
import glob
import os
from datetime import datetime

# Show all rows
pd.set_option('display.max_rows', None)

# Show all columns
pd.set_option('display.max_columns', None)
# Set options to show all rows and columns
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', None) # Prevents text inside cells from being cut off


rows_list = []
cols_to_remove = ['tif_paths', 'id', 'pred_paths']

for file in csv_files:
    filename = os.path.basename(file)
    c_time = os.path.getctime(file)
    dt_c_time = datetime.fromtimestamp(c_time)
    df = pd.read_csv(file)
    
    if len(df) >= 80:
        mean_row = df.iloc[[79]].copy()
        clean_row = mean_row.drop(columns=cols_to_remove, errors='ignore')
        
        # --- PARSING FILENAME ---
        clean_row['Losses'] = filename.split('_DA')[0] if '_DA' in filename else  filename.split('_epoch')[0]
        clean_row['epoch'] = filename.split('epoch_')[-1]
        clean_row['Train DA'] = filename.split('_DA')[-1].split('_')[0] if '_DA' in filename else  filename.split('_epoch')[0][-4:]
        clean_row['tta_status'] = 'no_TTA' if 'no_TTA' in filename else 'TTA'
        
        if 'overlap_' in filename:
            overlap_val = filename.split('overlap_')[1].split('_')[0]
            clean_row['overlap'] = float(overlap_val)
            
        if 'th_' in filename:
            th_val = filename.split('th_')[1].replace('.csv', '')
            clean_row['threshold'] = float(th_val)
        clean_row['dt_c_time'] = dt_c_time

        if 'post_process_' in str(file).split('/')[-2]:
            clean_row['postprocess'] = str(file).split('/')[-2].split('post_process_')[-1]
        else:
            clean_row['postprocess'] = 'no_postprocess'
        rows_list.append(clean_row)

# 2. Create the new combined DataFrame


combined_df = pd.concat(rows_list, ignore_index=True)

# 3. REORDER COLUMNS: Define the leading order
# We list exactly what we want first, then add the rest of the columns
fixed_cols = ['Losses', 'epoch', 'Train DA', 'tta_status', 'overlap', 'threshold', 'postprocess']
other_cols = [c for c in combined_df.columns if c not in fixed_cols]

combined_df = combined_df[fixed_cols + other_cols]

# 4. View the result
combined_df.sort_values(['image_score'], ascending=False)
#combined_df.sort_values(['Losses', 'epoch', 'tta_status', 'image_score'], ascending=False)

,Losses,epoch,Train DA,tta_status,overlap,threshold,postprocess,image_score,topo_score,surface_dice,voi_score,voi_split,voi_merge,dt_c_time
127,BCESoftDiceclDiceLoss,Final_5_TTA_overlap_0.5_th_0.5.csv,Loss,TTA,0.5,0.5,hysteresis_TH_propragation_0.45_0.8_iters2,0.610353,0.352847,0.879463,0.561962,1.326105,1.319880,2026-02-27 00:14:02.051761
128,BCESoftDiceclDiceLoss,Final_5_TTA_overlap_0.5_th_0.5.csv,Loss,TTA,0.5,0.5,hysteresis_TH_propragation_0.45_0.75_iters2,0.610284,0.352650,0.879506,0.561892,1.327093,1.319734,2026-02-26 17:11:57.450311
118,BCESoftDiceclDiceLoss,30_4_TTA_overlap_0.5_th_0.5.csv,Loss,TTA,0.5,0.5,hysteresis_TH_propragation_0.45_0.85_iters2,0.608301,0.348840,0.877568,0.561430,1.320478,1.330085,2026-02-25 21:31:05.952077
119,BCESoftDiceclDiceLoss,30_4_TTA_overlap_0.5_th_0.5.csv,Loss,TTA,0.5,0.5,hysteresis_TH_propragation_0.5_0.85_iters2,0.608084,0.351133,0.875849,0.560564,1.334985,1.328229,2026-02-26 02:27:40.961095
117,BCESoftDiceclDiceLoss,30_4_TTA_overlap_0.5_th_0.5.csv,Loss,TTA,0.5,0.5,hysteresis_TH_propragation_0.45_0.8_iters2,0.607558,0.346369,0.877798,0.561193,1.323892,1.329541,2026-02-25 16:33:26.265830
120,BCE_Twersky,Final_4_TTA_overlap_0.5_th_0.5.csv,rsky,TTA,0.5,0.5,hysteresis_TH_propragation_0.45_0.8_iters2,0.605879,0.347344,0.859752,0.573608,1.163502,1.348339,2026-02-26 07:12:10.350106
121,BCE_Twersky,Final_4_TTA_overlap_0.5_th_0.5.csv,rsky,TTA,0.5,0.5,hysteresis_TH_propragation_0.45_0.75_iters2,0.605760,0.347044,0.859718,0.573557,1.164192,1.348238,2026-02-26 12:14:28.001998
116,BCESoftDiceclDiceLoss,Final_3_TTA_overlap_0.5_th_0.5.csv,Loss,TTA,0.5,0.5,hysteresis_TH_propragation_0.45_0.8_iters2,0.604976,0.343423,0.872454,0.561686,1.316199,1.331367,2026-02-25 03:28:07.716651
125,BCE_Twersky,Final_3_TTA_overlap_0.5_th_0.5.csv,rsky,TTA,0.5,0.5,hysteresis_TH_propragation_0.45_0.8_iters2,0.603077,0.340916,0.856667,0.574195,1.141971,1.362069,2026-02-24 20:53:09.495928
126,BCE_Twersky,Final_3_TTA_overlap_0.5_th_0.5.csv,rsky,TTA,0.5,0.5,hysteresis_TH_propragation_0.45_0.75_iters2,0.602865,0.340342,0.856622,0.574127,1.142828,1.361926,2026-02-25 08:10:57.310284


In [16]:

# Save to CSV
combined_df.to_csv("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/csv_results/resume.csv", index=False)

In [13]:

import pandas as pd

# Basic save
combined_df.to_csv('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/resume_results_old.csv', index=False)

# check individual results

In [14]:
df_1 = pd.read_csv("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/BCE_Tversky_DAsafe_epoch_model_best_TTA_overlap_0.5_th_0.5.csv")
df_1.sort_values("topo_score", ascending=False)

,tif_paths,id,pred_paths,image_score,topo_score,surface_dice,voi_score,voi_split,voi_merge
15,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,3694884789,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.693013,0.529082,0.931205,0.595334,1.173808,1.091960
76,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,2821927657,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.634440,0.383840,0.924373,0.559309,1.437412,1.188995
45,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,3527825464,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.605481,0.335926,0.923863,0.518145,1.666618,1.433251
71,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,1127903126,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.536690,0.325816,0.759534,0.494595,1.985904,1.420284
14,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,378779937,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.598691,0.319782,0.919201,0.517247,1.670369,1.440673
30,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,1845578058,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.630130,0.319326,0.939659,0.587004,1.244829,1.100387
38,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,3515764944,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.625492,0.312308,0.941122,0.578304,1.287822,1.142826
19,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,2961547523,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.611297,0.298507,0.929317,0.561383,1.255832,1.348549
51,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,363381119,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.608135,0.296670,0.907624,0.575617,1.086349,1.371210
41,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,1435658104,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.599183,0.289979,0.908507,0.554891,1.472123,1.201730


In [15]:
df_2 = pd.read_csv(
    "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_dilated/results_post.csv")
df_2[df['id'] == "2961547523"]

FileNotFoundError: [Errno 2] No such file or directory: '/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_dilated/results_post.csv'

In [ ]:
# Merge the two dataframes on the 'id' column
combined_df = df_1.merge(df_2[['id', 'voi_score']], on='id', suffixes=('_1', '_2'))

combined_df['diff'] = combined_df['voi_score_1'] - combined_df['voi_score_2']
combined_df = combined_df.sort_values('diff', ascending=False)
combined_df = combined_df[['id', 'voi_score_1', 'voi_score_2', 'diff']]
print(f"mean: {combined_df['diff'].mean()}")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(combined_df)



# Copy all csv to an independent folder

In [ ]:
from pathlib import Path
import shutil

# Define directories
base_path = Path("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences")
dest_root = Path("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/csv_results")

# Find all .csv files
csv_files = list(base_path.rglob("*.csv"))

for file in csv_files:
    file_str = str(file)
    
    # Filtering logic
    if '/tests/' in file_str or file.name.startswith('th_'):
        continue
    
    # Create the new path using pathlib (cleaner than .replace)
    relative_path = file.relative_to(base_path)
    dest_path = dest_root / relative_path

    # Create the folder structure in the destination root
    dest_path.parent.mkdir(parents=True, exist_ok=True)
    
    # Copy the file
    shutil.copy2(file, dest_path)
    print(f"Copied: {dest_path}")